## **Extracción de datos de la REST API de XM S.A. E.S.P.**

En 2020, XM S.A. E.S.P. puso a disposición del público su API REST para facilitar la consulta de datos del Sistema Eléctrico Colombiano. Para simplificar aún más el uso de la API, desarrollaron un módulo llamado *pydataxm.py*, que permite realizar consultas a la API de manera más directa y sencilla, devolviendo un *DataFrame* de Pandas con los datos de cada variable consultada.

Utilizaremos esta API REST para extraer los datos necesarios para nuestro análisis.

In [1]:
import pandas as pd
import datetime as dt
import calendar
import requests
import json

In [2]:
import pydataxm as api_xm
consult = api_xm.ReadDB()

## Funciones de Apoyo

La API REST de XM tiene un límite máximo de 31 días (registros) por consulta. Para evitar errores al superar este límite, se han creado las siguientes funciones, las cuales permiten descargar un mes a la vez.

In [3]:
def generate_date_dict(start_year, end_year, end_month):
    date_dict = {}
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == end_year and month > end_month:
                break
            start_date = dt.date(year, month, 1)
            end_date = dt.date(year, month, calendar.monthrange(year, month)[1])
            date_dict[(year, month)] = {"start_date": start_date, "end_date": end_date}
    return date_dict

def extract_metric(date_dict, metric, num_metric):
    df_to_concat = pd.DataFrame()
    for key in date_dict:
        start_date = date_dict[key]['start_date']
        end_date = date_dict[key]['end_date']
        print(f"Start for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df = consult.request_data(metric, num_metric, start_date, end_date)
        print(f"Done for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df_to_concat = pd.concat([df_to_concat, df], ignore_index=True)
    return df_to_concat

## Extracción de Listados

### Listado de Metricas

Estas son las métricas o variables disponibles para extraccion con la API.

In [18]:
url = "http://servapibi.xm.com.co/lists"
request = {
    "MetricId": "ListadoMetricas",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_metricas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_metricas

,Id,Values.MetricId,Values.MetricName,Values.Entity,Values.MaxDays,Values.Type,Values.Url,Values.Filter,Values.MetricUnits,Values.MetricDescription
0,Sistema,DemaReal,Demanda Real por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Demanda de usuarios regulados y no regulados q...
1,Sistema,DemaReal,Demanda Real por Agente,Agente,31,HourlyEntities,http://servapibi.xm.com.co/hourly,Codigo Comercializador,kWh,Demanda de usuarios regulados y no regulados q...
2,Sistema,ExpoMoneda,Exportaciones Moneda por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,COP,Transferencias de Energia desde Colombia hacia...
3,Sistema,DemaCome,Demanda Comercial por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Considera la demanda propia de cada comerciali...
4,Sistema,Gene,Generación por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Generacion neta de cada una de las plantas Nac...
...,...,...,...,...,...,...,...,...,...,...
182,Sistema,ListadoAgentes,Listado de agentes con atributos por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de Agentes registrados ante el ASIC co...
183,Sistema,ListadoRios,Listado de rios por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de ríos que se encuentran registrados ...
184,Sistema,ListadoEmbalses,Listado de embalses por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de embalses que se encuentran registra...
185,Sistema,ListadoMetricas,Listado de Métricas por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de métricas disponibles en la API XM c...


In [19]:
listado_metricas.to_json('./data/listado_metricas.json', orient='records', lines=True)

### Listado de Plantas de Generación

In [20]:
request = {
    "MetricId": "ListadoRecursos",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_plantas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_plantas

,Id,Values.Code,Values.Name,Values.Type,Values.Disp,Values.RecType,Values.CompanyCode,Values.EnerSource,Values.OperStartdate,Values.State
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION
...,...,...,...,...,...,...,...,...,...,...
680,Sistema,VNTB,VENTANA B,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,HDTG,AGUA,1957-11-01,OPERACION
681,Sistema,ZPA2,ZIPAEMG 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1964-01-01,OPERACION
682,Sistema,ZPA3,ZIPAEMG 3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1976-01-01,OPERACION
683,Sistema,ZPA4,ZIPAEMG 4,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1981-04-01,OPERACION


In [21]:
listado_plantas.to_json('./data/listado_recursos_plantas.json', orient='records', lines=True)

### Listado de Ríos

In [22]:
request = {
    "MetricId": "ListadoRios",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_rios = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_rios

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,ALICBOGO,BOGOTA N.R.,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,AMOYAMOY,AMOYA,CENTRO,ACTIVO
3,Sistema,BAJOANCH,DIGUA,VALLE,ACTIVO
4,Sistema,BETAMAG1,BETANIA CP,CENTRO,ACTIVO
5,Sistema,BOCATENC,CONCEPCION,ANTIOQUIA,ACTIVO
6,Sistema,CAL1CALM,CALIMA,VALLE,ACTIVO
7,Sistema,CAMECAMP,CAMPOALEGRE,CALDAS,ACTIVO
8,Sistema,CAMECHIN,CHINCHINA,CALDAS,ACTIVO
9,Sistema,CLLRPORC,CARLOS LLERAS,ANTIOQUIA,ACTIVO


In [23]:
listado_rios.to_json('./data/listado_rios.json', orient='records', lines=True)

### Listado Embalses

In [24]:
request = {
    "MetricId": "ListadoEmbalses",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_embalses = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_embalses

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,AGREGADO,AGREGADO BOGOTA,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,BETANIA,BETANIA,CENTRO,ACTIVO
3,Sistema,CALIMA1,CALIMA1,VALLE,ACTIVO
4,Sistema,CHUZA,CHUZA,ORIENTE,ACTIVO
5,Sistema,ELQUIMBO,EL QUIMBO,CENTRO,ACTIVO
6,Sistema,ESMERALD,ESMERALDA,ORIENTE,ACTIVO
7,Sistema,FLORIDA2,FLORIDA II,RIOS ESTIMADOS,ACTIVO
8,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO
9,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO


In [25]:
listado_embalses.to_json('./data/listado_embalses.json', orient='records', lines=True)

### Listado Empresas del Sistema Electrico

In [27]:
request = {
    "MetricId": "ListadoAgentes",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_agentes = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_agentes

,Id,Values.Code,Values.Name,Values.Activity,Values.OperStartdate,Values.State
0,Sistema,GREC,GRUPO ENERGÉTICO SA ESP,COMERCIALIZACIÓN,2024-05-15,OPERACION
1,Sistema,SDEG,SOLAR EL EDÉN S.A.S. E.S.P.,GENERACIÓN,2024-05-07,OPERACION
2,Sistema,CPSG,EL CAMPANO SOLAR S.A.S. E.S.P.,GENERACIÓN,2024-04-30,OPERACION
3,Sistema,SHAG,OPERADORA SHANGRI-LA S.A.S E.S.P,GENERACIÓN,2024-04-30,OPERACION
4,Sistema,CSMG,SOLAR VILLAVIEJA S.A.S. E.S.P.,GENERACIÓN,2024-04-29,OPERACION
...,...,...,...,...,...,...
612,Sistema,CTGC,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
613,Sistema,CTGD,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,DISTRIBUCIÓN,1995-07-20,INACTIVO
614,Sistema,EPPC,EMPRESAS PUBLICAS DE PEREIRA S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
615,Sistema,EPMT,EMPRESAS PUBLICAS DE MEDELLIN E.S.P.,TRANSPORTE,1995-01-01,OPERACION


In [29]:
listado_agentes.to_json('./data/listado_empresas_agentes.json', orient='records', lines=True)

### Listado de Autogeneradores

In [32]:
request = {
    "MetricId": "ListadoAGPE",
    "Entity": "Agente"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_autogeneradores = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_autogeneradores

,Id,Values.AgpeID,Values.Name,Values.CompanyCode,Values.ComZone,Values.VoltageLevel,Values.EnerSource,Values.InstalledCapacity
0,Agente,Agp00001,PROMOTORA RD LTDA,CMMC,CARIBE MAR,2,RAD SOLAR,9.000000
1,Agente,Agp00002,ORGANIZACION TERPEL S A,EPSC,VALLE DEL CAUCA,2,GAS,16.011109
2,Agente,Agp00003,ANDREA BARRIOS,CNSC,NORTE DE SANTANDER,1,RAD SOLAR,5.000000
3,Agente,Agp00004,JUAN JOSE SANTA VALLA,CMMC,CARIBE MAR,2,RAD SOLAR,10.000000
4,Agente,Agp00005,LUIS ALFONSO VILLAMIL VARGAS,CSSC,CARIBE SOL,1,RAD SOLAR,3.000000
...,...,...,...,...,...,...,...,...
8787,Agente,Grp0988,AGPE FUNDACION OFTALMOLOGICA,BIAC,NARIÑO,2,RAD SOLAR,20.000000
8788,Agente,Grp0993,AGPE RAMON GIRALDO GONZALEZ,SCEC,CARIBE MAR,1,RAD SOLAR,27.000000
8789,Agente,Unt0499,AGPE - ECOPETROL LA HORMIGA,GECG,BAJO PUTUMAYO,3,GAS,1000.000000
8790,Agente,Unt0514,AGPE ENTREPALMAS,EPSG,META,2,BIOGAS,500.000000


In [47]:
listado_autogeneradores.to_json('./data/listado_autogeneradores.json', orient='records', lines=True)

## Extracción de Variables de Generación

### Generación del Sistema por Hora

In [46]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "Gene"
num_metric = 0
df_generacion_sistema = extract_metric(date_dict,metric,num_metric)
df_generacion_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23000,5296750.71000,4919467.58000,4638776.22000,4442074.76000,4279026.64000,3827254.47000,3832573.95000,...,4391372.59000,4365640.95000,4605721.12000,6021430.06000,6527391.35000,6261393.45000,5776398.57000,5211439.50000,4714078.01000,2010-01-01
1,Sistema,Sistema,4443442.83000,4238022.31000,4153685.87000,4103574.79000,4153986.60000,4252501.43000,4185993.85000,4479587.60000,...,5397357.88000,5317214.52000,5482854.99000,6895377.37000,7356071.75000,7109606.16000,6611477.08000,5931707.62000,5211930.81000,2010-01-02
2,Sistema,Sistema,4771417.74000,4520041.29000,4364500.03000,4210941.04000,4205070.55000,4312551.86000,4156670.92000,4331637.24000,...,5015866.10000,4981069.04000,5169045.72000,6637669.36000,7126930.30000,6924827.71000,6432570.74000,5752050.08000,5135978.48000,2010-01-03
3,Sistema,Sistema,4705250.02000,4493314.06000,4399068.67000,4395569.56000,4475996.61000,4777750.57000,4911096.21000,5322214.49000,...,6618343.03000,6458778.34000,6529860.50000,7819390.86000,8232402.15000,7884258.05000,7197982.57000,6288573.58000,5535400.79000,2010-01-04
4,Sistema,Sistema,5171964.92000,4939629.25000,4840107.60000,4774763.87000,4949684.01000,5195571.83000,5362721.47000,5742326.06000,...,6778398.35000,6715045.20000,6745497.24000,8023899.14000,8336888.23000,7929022.67000,7243496.48000,6378988.75000,5737144.04000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80000,8198137.14000,7966493.50000,7848538.15000,7993954.30000,8373795.23000,8533971.50000,8989609.97000,...,10224371.17000,10138968.61000,9944771.52000,10267157.48000,10388113.98000,10186516.32000,9774795.94000,9269233.15000,8783850.22000,2024-04-26
5230,Sistema,Sistema,8416268.81000,8139684.03000,7934365.14000,7801216.87000,7802711.66000,7866953.71000,8060693.66000,8611512.50000,...,9613448.98000,9515583.90000,9453804.19000,10022279.38000,10070282.41000,9806988.02000,9391935.06000,8934686.71000,8515651.18000,2024-04-27
5231,Sistema,Sistema,8153358.39000,7823646.09000,7495560.68000,7290789.06000,7186795.65000,7058942.05000,6899473.59000,7167562.93000,...,8373980.81000,8391670.64000,8366776.05000,9037799.05000,9296771.60000,9194946.81000,8963686.66000,8545982.69000,8101750.18000,2024-04-28
5232,Sistema,Sistema,7755006.39000,7559477.81000,7424995.07000,7358043.41000,7541190.63000,7993483.44000,8218854.74000,8737461.99000,...,10365625.70000,10314129.94000,10040895.25000,10521997.09000,10740452.13000,10521840.29000,10125689.57000,9473347.41000,8842359.25000,2024-04-29


In [48]:
df_generacion_sistema.to_json('./data/generacion_sistema.json', orient='records', lines=True)

### Generacion por Planta Recurso

In [49]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "Gene"
num_metric = 1
df_generacion_planta_recurso = extract_metric(date_dict,metric,num_metric)
df_generacion_planta_recurso

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,AFR1,4116,4027,4069,4109,3990,4158,3887,3781,...,3814,3673,3754,3803,3828,3581,3679,3803,3613,2010-01-01
1,Recurso,ALBG,14824,14817,14804,14804,14813,13686,13098,17048,...,11080,11071,11055,11069,11086,11096,11098,11091,11049,2010-01-01
2,Recurso,AMF1,622,622,622,622,622,620,620,621,...,634,634,633,632,633,632,632,633,633,2010-01-01
3,Recurso,AMR1,2,3,5,3,,,,31,...,,,,,,,,,,2010-01-01
4,Recurso,ASN1,486,486,490,484,488,485,488,487,...,487,486,486,487,487,487,488,487,487,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817316,Recurso,VNTB,986.33,888.42,1065.41,704.19,1291.19,1545.66,1599.5,1662.4,...,2406.07,2389.54,2385.53,2357.29,2291.29,2340.3,2349.94,2317.12,2028.55,2024-04-30
817317,Recurso,ZPA2,29919.99,29953.03,29911.08,29938.3,29881.28,29941.76,29883,29952.68,...,29559.6,29554.02,29579.26,29572.7,29464.75,28800.29,29303.37,29623.16,29640.7,2024-04-30
817318,Recurso,ZPA3,,,,,,,,,...,,,,,,2829.99,17682.36,25107.14,30922.03,2024-04-30
817319,Recurso,ZPA4,60652.2,60717.28,60783.27,60703.95,61031.19,61028.64,61098.96,61142.5,...,61592.99,61363.92,61480.96,61260.58,61578.56,61480.5,61023.61,61048.03,61602.61,2024-04-30


In [50]:
df_generacion_planta_recurso.to_json('./data/generacion_planta_recurso.json', orient='records', lines=True)

### Generación Ideal Sistema

In [56]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "GeneIdea"
num_metric = 0
df_generacion_ideal_sistema = extract_metric(date_dict,metric,num_metric)
df_generacion_ideal_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23000,5299387.09000,4922337.76000,4640952.55000,4443431.20000,4280072.11000,3828774.82000,3832979.13000,...,4392140.29000,4366427.63000,4606016.29000,6021430.06000,6527391.35000,6263172.04000,5779281.34000,5212138.25000,4714247.65000,2010-01-01
1,Sistema,Sistema,4445348.51000,4238116.21000,4154887.32000,4103709.79000,4153986.60000,4252501.43000,4186498.45000,4480615.46000,...,5397357.88000,5318190.02000,5484348.49000,6895377.37000,7356107.90000,7110891.37000,6612818.22000,5934135.44000,5214100.60000,2010-01-02
2,Sistema,Sistema,4772268.38000,4520104.69000,4365723.51000,4211679.57000,4205364.41000,4312551.86000,4156670.92000,4333251.97000,...,5016125.95000,4981354.24000,5169667.20000,6637669.36000,7126930.30000,6927793.22000,6433970.64000,5753971.20000,5137059.49000,2010-01-03
3,Sistema,Sistema,4705845.03000,4493923.68000,4400230.86000,4396643.54000,4475996.61000,4778560.19000,4912357.20000,5323877.68000,...,6517107.95000,6357582.34000,6428430.50000,7712972.01000,8130900.15000,7782972.05000,7096426.57000,6186918.31000,5434024.79000,2010-01-04
4,Sistema,Sistema,5044349.31000,4810224.85000,4711179.46000,4644547.47000,4821049.57000,5066868.38000,5233620.74000,5612777.12000,...,6648597.90000,6586555.45000,6617438.28000,7888852.40000,8206986.44000,7800736.46000,7115549.68000,6250156.98000,5605618.05000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80000,8198137.14000,7966493.50000,7848538.15000,7993954.30000,8373795.23000,8533971.50000,8989609.97000,...,10224371.17000,10138968.61000,9944771.52000,10267157.48000,10388113.98000,10186516.32000,9774795.94000,9269233.15000,8783850.22000,2024-04-26
5230,Sistema,Sistema,8416268.81000,8139684.03000,7934365.14000,7801216.87000,7802711.66000,7866953.71000,8060693.66000,8611512.50000,...,9613448.98000,9515583.90000,9453804.19000,10022279.38000,10070282.41000,9806988.02000,9391935.06000,8934686.71000,8515651.18000,2024-04-27
5231,Sistema,Sistema,8153358.39000,7823646.09000,7495560.68000,7290789.06000,7186795.65000,7058942.05000,6899473.59000,7167562.93000,...,8373980.81000,8391670.64000,8366776.05000,9037799.05000,9296771.60000,9195967.42000,8965875.21000,8547088.54000,8101750.18000,2024-04-28
5232,Sistema,Sistema,7755006.39000,7559477.81000,7424995.07000,7358043.41000,7541190.63000,7993483.44000,8218854.74000,8737461.99000,...,10365625.70000,10315355.57000,10040895.25000,10521997.09000,10741050.68000,10522650.66000,10127382.98000,9476883.76000,8842359.25000,2024-04-29


In [57]:
df_generacion_ideal_sistema.to_json('./data/generacion_ideal_sistema.json', orient='records', lines=True)

### Generacion Ideal Planta Recurso

In [60]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "GeneIdea"
num_metric = 1
df_generacion_ideal_planta_recurso = extract_metric(date_dict,metric,num_metric)
df_generacion_ideal_planta_recurso

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,AFR1,4116,4027,4069,4109,3990,4158,3887,3781,...,3814,3673,3754,3803,3828,3581,3679,3803,3613,2010-01-01
1,Recurso,ALBG,11000,11000,11000,11000,11000,11000,11000,11000,...,11000,11000,11000,11000,11000,11000,11000,11000,11000,2010-01-01
2,Recurso,AMF1,622,622,622,622,622,620,620,621,...,634,634,633,632,633,632,632,633,633,2010-01-01
3,Recurso,AMR1,2,3,5,3,,,,31,...,,,,,,,,,,2010-01-01
4,Recurso,ASN1,486,486,490,484,488,485,488,487,...,487,486,486,487,487,487,488,487,487,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804266,Recurso,VLT1,10038.6,10010,9873.6,9015.6,8199.4,9752.6,10060.6,10025.4,...,10065,10828.4,10608.4,10538,10164,8329.2,4833.4,5537.4,4496.8,2024-04-30
804267,Recurso,VNTB,986.33,888.42,1065.41,704.19,1291.19,1545.66,1599.5,1662.4,...,2406.07,2389.54,2385.53,2357.29,2291.29,2340.3,2349.94,2317.12,2028.55,2024-04-30
804268,Recurso,ZPA2,28000,26000,24000,9000,,,,,...,,,,,,,,,,2024-04-30
804269,Recurso,ZPA4,59000,57000,55000,53000,51000,49000,47000,16000,...,,,,,,,,,,2024-04-30


In [61]:
df_generacion_ideal_planta_recurso.to_json('./data/generacion_ideal_planta_recurso.json', orient='records', lines=True)

### Perdidas del Sistema

In [4]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "PerdidasEner"
num_metric = 0
df_perdidas_sistema = extract_metric(date_dict,metric,num_metric)
df_perdidas_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,105774.61000,102118.61000,83645.43000,76479.29000,86936.10000,90440.22000,63824.36000,58145.23000,...,91227.73000,72140.80000,81412.93000,101288.44000,117722.66000,105167.30000,117369.59000,117319.88000,63782.69000,2010-01-01
1,Sistema,Sistema,67249.86000,68144.93000,80334.54000,71838.53000,82755.38000,73647.53000,57781.19000,60754.75000,...,76224.22000,73456.17000,80452.90000,82028.96000,123472.06000,135808.67000,136890.04000,153906.03000,138367.05000,2010-01-02
2,Sistema,Sistema,85150.73000,82398.64000,72624.72000,68972.34000,94164.66000,91730.56000,63489.63000,77847.16000,...,103027.98000,99768.55000,87914.68000,91854.79000,156818.60000,148016.20000,149224.20000,147846.70000,148218.54000,2010-01-03
3,Sistema,Sistema,105862.54000,105914.33000,78999.12000,73351.31000,46846.53000,69821.14000,58761.25000,57610.79000,...,114344.91000,69098.95000,63711.63000,74057.35000,128479.40000,145726.62000,146592.46000,114760.41000,41776.99000,2010-01-04
4,Sistema,Sistema,76579.31000,95728.29000,86616.09000,63866.76000,79934.42000,66618.63000,59586.27000,58142.88000,...,87785.94000,82425.43000,58046.85000,128688.74000,122294.09000,110237.18000,112819.66000,135709.39000,216910.07000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,126262.35000,125898.61000,125845.56000,116964.45000,100126.47000,112034.68000,125497.24000,129908.44000,...,169304.68000,188228.31000,245061.68000,143956.22000,152486.43000,158102.01000,154361.52000,141111.41000,145943.10000,2024-04-26
5230,Sistema,Sistema,145872.60000,142324.34000,140287.40000,134343.81000,128968.31000,130514.03000,118078.49000,132046.72000,...,137922.41000,146466.64000,146785.72000,150258.12000,166424.90000,164811.00000,156758.48000,138097.20000,126882.77000,2024-04-27
5231,Sistema,Sistema,131308.39000,125172.03000,125330.72000,126857.51000,124666.49000,118256.46000,106743.08000,108166.40000,...,126654.41000,117944.98000,108010.41000,122707.43000,129748.61000,129918.48000,129930.01000,117732.00000,110637.54000,2024-04-28
5232,Sistema,Sistema,97878.18000,96572.57000,95367.40000,95241.84000,78463.46000,95768.18000,101123.09000,101598.94000,...,128655.82000,142992.37000,127562.01000,136332.37000,151891.98000,148351.96000,154116.17000,133460.55000,113115.72000,2024-04-29


In [5]:
df_perdidas_sistema.to_json('./data/perdidas_sistema.json', orient='records', lines=True)

## Extracción de Variables Capacidad y Volumen

### Volumen Util del Sistema en Unidades de Energía

In [64]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "VoluUtilDiarEner"
num_metric = 0
df_volumen_util_energia_sistema = extract_metric(date_dict,metric,num_metric)
df_volumen_util_energia_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,10006614779.00000,2010-01-01
1,Sistema,9995146811.50000,2010-01-02
2,Sistema,9992677303.10000,2010-01-03
3,Sistema,9965341213.60000,2010-01-04
4,Sistema,9926759570.30000,2010-01-05
...,...,...,...
5229,Sistema,5399083900.00000,2024-04-26
5230,Sistema,5454064900.00000,2024-04-27
5231,Sistema,5557993200.00000,2024-04-28
5232,Sistema,5672509900.00000,2024-04-29


In [65]:
df_volumen_util_energia_sistema.to_json('./data/volumen_util_energia_sistema.json', orient='records', lines=True)

### Volumen Util por Planta Recurso en Unidades de Energía

In [66]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "VoluUtilDiarEner"
num_metric = 1
df_volumen_util_energia_planta = extract_metric(date_dict,metric,num_metric)
df_volumen_util_energia_planta

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Embalse,AGREGADO BOGOTA,2.046298e+09,2010-01-01
1,Embalse,ALTOANCHICAYA,1.712760e+07,2010-01-01
2,Embalse,AMANI,2.007019e+08,2010-01-01
3,Embalse,BETANIA,1.252323e+08,2010-01-01
4,Embalse,CALIMA1,1.722852e+08,2010-01-01
...,...,...,...,...
116514,Embalse,SALVAJINA,2.291914e+08,2024-04-30
116515,Embalse,SAN LORENZO,3.830000e+07,2024-04-30
116516,Embalse,TOPOCORO,3.250895e+08,2024-04-30
116517,Embalse,TRONERAS,2.472430e+07,2024-04-30


In [67]:
df_volumen_util_energia_planta.to_json('./data/volumen_util_energia_planta.json', orient='records', lines=True)

### Capacidad Util Sistema Unidades Energia

In [70]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapaUtilDiarEner"
num_metric = 0
df_capacidad_util_energia = extract_metric(date_dict,metric,num_metric)
df_capacidad_util_energia

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,15428396811.00000,2010-01-01
1,Sistema,15428396811.00000,2010-01-02
2,Sistema,15428396811.00000,2010-01-03
3,Sistema,15428396811.00000,2010-01-04
4,Sistema,15428396811.00000,2010-01-05
...,...,...,...
5229,Sistema,17359561885.00000,2024-04-26
5230,Sistema,17359561885.00000,2024-04-27
5231,Sistema,17359561885.00000,2024-04-28
5232,Sistema,17359561885.00000,2024-04-29


In [71]:
df_capacidad_util_energia.to_json('./data/capacidad_util_energia.json', orient='records', lines=True)

### Capacidad Util Energia por Embalse

In [74]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapaUtilDiarEner"
num_metric = 1
df_capacidad_util_energia_embalse = extract_metric(date_dict,metric,num_metric)
df_capacidad_util_energia_embalse

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Embalse,AGREGADO BOGOTA,4.447220e+09,2010-01-01
1,Embalse,ALTOANCHICAYA,3.721801e+07,2010-01-01
2,Embalse,AMANI,2.303661e+08,2010-01-01
3,Embalse,BETANIA,1.426662e+08,2010-01-01
4,Embalse,CALIMA1,2.035201e+08,2010-01-01
...,...,...,...,...
116656,Embalse,SALVAJINA,5.295982e+08,2024-04-30
116657,Embalse,SAN LORENZO,4.251428e+08,2024-04-30
116658,Embalse,TOPOCORO,1.033507e+09,2024-04-30
116659,Embalse,TRONERAS,6.384768e+07,2024-04-30


In [75]:
df_capacidad_util_energia_embalse.to_json('./data/capacidad_util_energia_embalse.json', orient='records', lines=True)

### Capacidad Efectiva Neta por Recurso

In [82]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapEfecNeta"
num_metric = 1
df_capacidad_efectiva_neta = extract_metric(date_dict,metric,num_metric)
df_capacidad_efectiva_neta

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Code,Value,Date
0,Recurso,2QFU,4000.00000,2010-01-01
1,Recurso,ABJ1,450.00000,2010-01-01
2,Recurso,AFR1,7050.00000,2010-01-01
3,Recurso,ALBG,429000.00000,2010-01-01
4,Recurso,AMF1,810.00000,2010-01-01
...,...,...,...,...
1153572,Recurso,VNTB,2500.00000,2024-04-30
1153573,Recurso,ZPA2,36000.00000,2024-04-30
1153574,Recurso,ZPA3,63000.00000,2024-04-30
1153575,Recurso,ZPA4,64000.00000,2024-04-30


In [83]:
df_capacidad_efectiva_neta.to_json('./data/capacidad_efectiva_neta.json', orient='records', lines=True)

### Aportes de Energia del Sistema

In [86]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEner"
num_metric = 0
df_aportes_energia_sistema = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,45616100.00000,2010-01-01
1,Sistema,43097100.00000,2010-01-02
2,Sistema,43400400.00000,2010-01-03
3,Sistema,40410400.00000,2010-01-04
4,Sistema,41190600.00000,2010-01-05
...,...,...,...
5229,Sistema,175197100.00000,2024-04-26
5230,Sistema,190925700.00000,2024-04-27
5231,Sistema,206187700.00000,2024-04-28
5232,Sistema,303206200.00000,2024-04-29


In [87]:
df_aportes_energia_sistema.to_json('./data/aportes_energia_sistema.json', orient='records', lines=True)

### Aportes de Energia del Sistema por cada Río

In [90]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEner"
num_metric = 1
df_aportes_energia_sistema_rio = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema_rio

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,4352200.0,2010-01-01
1,Rio,ALTOANCHICAYA,3209600.0,2010-01-01
2,Rio,BATA,1461000.0,2010-01-01
3,Rio,CAUCA SALVAJINA,2867400.0,2010-01-01
4,Rio,CHUZA,296100.0,2010-01-01
...,...,...,...,...
158563,Rio,SAN FRANCISCO,360300.0,2024-04-30
158564,Rio,SAN MIGUEL,1111500.0,2024-04-30
158565,Rio,SINU URRA,4322300.0,2024-04-30
158566,Rio,SOGAMOSO,17014300.0,2024-04-30


In [91]:
df_aportes_energia_sistema_rio.to_json('./data/aportes_energia_sistema_rio.json', orient='records', lines=True)

### Aportes Media Historica Sistema

In [98]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEnerMediHist"
num_metric = 0
df_aportes_energia_sistema_media_historica = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema_media_historica

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,75130000.00000,2010-01-01
1,Sistema,75130000.00000,2010-01-02
2,Sistema,75130000.00000,2010-01-03
3,Sistema,75130000.00000,2010-01-04
4,Sistema,75130000.00000,2010-01-05
...,...,...,...
5229,Sistema,222790000.00000,2024-04-26
5230,Sistema,222790000.00000,2024-04-27
5231,Sistema,222790000.00000,2024-04-28
5232,Sistema,222790000.00000,2024-04-29


In [99]:
df_aportes_energia_sistema_media_historica.to_json('./data/aportes_energia_sistema_media_historica.json', orient='records', lines=True)

### Aportes Media Historica Río

In [100]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEnerMediHist"
num_metric = 1
df_aportes_energia_rio_media_historica = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_rio_media_historica

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,5520000.0,2010-01-01
1,Rio,ALTOANCHICAYA,4370000.0,2010-01-01
2,Rio,BATA,2640000.0,2010-01-01
3,Rio,BLANCO,210000.0,2010-01-01
4,Rio,BOGOTA N.R.,3720000.0,2010-01-01
...,...,...,...,...
167232,Rio,SAN FRANCISCO,340000.0,2024-04-30
167233,Rio,SAN MIGUEL,1780000.0,2024-04-30
167234,Rio,SINU URRA,2820000.0,2024-04-30
167235,Rio,SOGAMOSO,19000000.0,2024-04-30


In [101]:
df_aportes_energia_rio_media_historica.to_json('./data/aportes_energia_rio_media_historica.json', orient='records', lines=True)

### Aportes Caudales Ríos

In [10]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'AporCaudal'
num_metric = 0
df_aportes_caudal = extract_metric(date_dict,metric,num_metric)
df_aportes_caudal

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,18.810,2010-01-01
1,Rio,ALTOANCHICAYA,31.470,2010-01-01
2,Rio,BATA,8.730,2010-01-01
3,Rio,BLANCO,0.000,2010-01-01
4,Rio,BOGOTA N.R.,0.000,2010-01-01
...,...,...,...,...
163711,Rio,SAN FRANCISCO,4.444,2024-04-30
163712,Rio,SAN MIGUEL,32.920,2024-04-30
163713,Rio,SINU URRA,374.030,2024-04-30
163714,Rio,SOGAMOSO,513.420,2024-04-30


In [11]:
df_aportes_caudal.to_json('./data/aportes_caudal.json', orient='records', lines=True)

### Aportes Caudales Rios Media Historica

In [9]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'AporCaudalMediHist'
num_metric = 0
df_aportes_caudal_media_historica = extract_metric(date_dict,metric,num_metric)
df_aportes_caudal_media_historica

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,23.87,2010-01-01
1,Rio,ALTOANCHICAYA,42.88,2010-01-01
2,Rio,BATA,15.76,2010-01-01
3,Rio,BLANCO,0.47,2010-01-01
4,Rio,BOGOTA N.R.,8.54,2010-01-01
...,...,...,...,...
163711,Rio,SAN FRANCISCO,4.14,2024-04-30
163712,Rio,SAN MIGUEL,52.64,2024-04-30
163713,Rio,SINU URRA,244.26,2024-04-30
163714,Rio,SOGAMOSO,573.25,2024-04-30


In [10]:
df_aportes_caudal_media_historica.to_json('./data/aportes_caudal_media_historica.json', orient='records', lines=True)

## Variables de Demanda

### Demanda Real del Sitema

In [4]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'DemaReal'
num_metric = 0
df_dema_real = extract_metric(date_dict,metric,num_metric)
df_dema_real

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5504207.62000,5197268.48000,4838692.33000,4564473.26000,4356495.10000,4189631.89000,3764950.46000,3774833.90000,...,4300912.56000,4294286.83000,4524603.36000,5920141.62000,6409668.69000,6158004.74000,5661911.75000,5094818.37000,4650464.96000,2010-01-01
1,Sistema,Sistema,4378098.65000,4169971.28000,4074552.78000,4031871.26000,4071231.22000,4178853.90000,4128717.26000,4419860.71000,...,5321133.66000,5244733.85000,5403895.59000,6813348.41000,7232635.84000,6975082.70000,6475928.18000,5780229.41000,5075733.55000,2010-01-02
2,Sistema,Sistema,4687117.65000,4437706.05000,4293098.79000,4142707.23000,4111199.75000,4220821.30000,4093181.29000,4255404.81000,...,4913097.97000,4881585.69000,5081752.52000,6545814.57000,6970111.70000,6779777.02000,6284746.44000,5606124.50000,4988840.95000,2010-01-03
3,Sistema,Sistema,4599982.49000,4388009.35000,4321231.74000,4323292.23000,4429150.08000,4708739.05000,4853595.95000,5266266.89000,...,6503998.12000,6389679.39000,6466148.87000,7745333.51000,8103922.75000,7738531.43000,7051390.11000,6173813.17000,5493623.80000,2010-01-04
4,Sistema,Sistema,5095385.61000,4843900.96000,4753491.51000,4710897.11000,4869749.59000,5128953.20000,5303135.20000,5684183.18000,...,6690612.41000,6632619.77000,6687450.39000,7895210.40000,8214594.14000,7818785.49000,7130676.82000,6243279.36000,5520233.97000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8391948.45000,8072238.53000,7840647.94000,7731573.70000,7893827.83000,8261760.55000,8408474.26000,8859701.53000,...,10055066.49000,9950740.30000,9699709.84000,10123201.26000,10235627.55000,10028414.31000,9620434.42000,9128121.74000,8637907.12000,2024-04-26
5230,Sistema,Sistema,8270396.21000,7997359.69000,7794077.74000,7666873.06000,7673743.35000,7736439.68000,7942615.17000,8479465.78000,...,9475526.57000,9369117.26000,9307018.47000,9872021.26000,9903857.51000,9642177.02000,9235176.58000,8796589.51000,8388768.41000,2024-04-27
5231,Sistema,Sistema,8022050.00000,7698474.06000,7370229.96000,7163931.55000,7062129.16000,6940685.59000,6792730.51000,7059396.53000,...,8247326.40000,8273725.66000,8258765.64000,8915091.62000,9167022.99000,9066048.94000,8835945.20000,8429356.54000,7991112.64000,2024-04-28
5232,Sistema,Sistema,7657128.21000,7462905.24000,7329627.67000,7262801.57000,7462727.17000,7897715.26000,8117731.65000,8635863.05000,...,10236969.88000,10172363.20000,9913333.24000,10385664.72000,10589158.70000,10374298.70000,9973266.81000,9343423.21000,8729243.53000,2024-04-29


In [5]:
df_dema_real.to_json('./data/dema_real.json', orient='records', lines=True)

### Demanda Real por Agente

In [4]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'DemaReal'
num_metric = 1
df_dema_real_agente = extract_metric(date_dict,metric,num_metric)
df_dema_real_agente

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Agente,ASCC,1267.81,1218.85,1264.56,1257.77,1218.03,1225.26,1169.77,1275.29,...,1551.8,1497.98,1492.23,1664.55,1650.29,1582.45,1538.11,1462.21,1410.2,2010-01-01
1,Agente,CALC,578.72,578.72,578.72,578.72,578.72,578.72,,,...,,,,578.72,578.72,578.72,578.72,578.72,578.72,2010-01-01
2,Agente,CASC,30764.76,28612.12,26446.25,24849.5,23835.07,22579.47,20944.84,21375.04,...,22761.89,22895.26,24746.31,34666.46,37462.27,34796.25,30597.94,27095.36,24934.21,2010-01-01
3,Agente,CCOC,129948,128913,131727.16,130930.73,130135.79,126604.39,124304.56,125475.24,...,147276.01,150356.99,148992.6,157035.76,153992.23,153360.99,148249.94,143997.5,137285,2010-01-01
4,Agente,CDIC,75735.12,67985.53,60026.22,52439.02,49313.72,46401.43,48497.13,46289.01,...,52004.74,51170.18,55982.33,85666.77,102309.6,91160.98,72779.19,48009.83,38140.08,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412772,Agente,TRPG,,,,,,,,,...,,,,,,,,,,2024-04-30
412773,Agente,TYPG,1281.94,1290.77,1283.24,1306.35,1327.49,1362.44,1359.83,1252.09,...,839.62,905.59,989.01,937.43,950.12,961.12,1027.25,1079.84,1151.9,2024-04-30
412774,Agente,UNGG,9.36,9.2,9.38,9.5,9.48,7.27,,,...,,,,6.19,9.98,8.8,8.67,8.9,9.12,2024-04-30
412775,Agente,VESC,2314.73,2274.55,2053.33,1773.45,1728.78,1878.52,1763.9,2151.62,...,918.02,1114.11,1158.37,1066.87,1023.3,1082.71,1143.5,900.15,766.96,2024-04-30


In [5]:
df_dema_real_agente.to_json('./data/dema_real_agente.json', orient='records', lines=True)

## Exportaciones e Importaciones

In [6]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'ExpoEner'
num_metric = 0
df_exportaciones = extract_metric(date_dict,metric,num_metric)
df_exportaciones

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,1215.36000,,,,,,,,...,,,,1529.10000,1545.03000,,,,,2010-01-01
1,Sistema,Sistema,,,,,2531.75000,3798.60000,,,...,240.29000,,,2542.24000,,,,,,2010-01-02
2,Sistema,Sistema,,,,,,468.09000,71.33000,,...,,,,2269.17000,725.77000,,,,,2010-01-03
3,Sistema,Sistema,,,,,631.25000,,,,...,99504.00000,98504.00000,98432.00000,105422.58000,99632.00000,97816.00000,99416.00000,99824.00000,98200.93000,2010-01-04
4,Sistema,Sistema,125750.00000,126928.00000,126648.00000,128496.00000,126536.00000,127072.00000,127680.00000,128248.00000,...,128136.00000,124624.00000,124104.00000,132904.00000,128016.00000,125208.00000,125222.51000,126120.00000,126608.00000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,Sistema,Sistema,,960.27000,2488.08000,1176.26000,1057.66000,575.29000,,892.90000,...,3022.69000,,5473.61000,3177.63000,2374.35000,,1215.58000,1950.76000,1519.32000,2024-04-18
4825,Sistema,Sistema,,4893.81000,2044.98000,1409.93000,1699.55000,997.52000,2426.08000,540.20000,...,,,,,,,,,,2024-04-19
4826,Sistema,Sistema,,,,,,,,,...,,,21.60000,8483.20000,2253.60000,,,,1679.82000,2024-04-28
4827,Sistema,Sistema,5693.70000,5695.14000,4883.83000,1600.46000,1961.09000,1927.06000,103.12000,2762.36000,...,1482.52000,,1972.76000,5644.73000,,,,,859.69000,2024-04-29


In [7]:
df_exportaciones.to_json('./data/exportaciones.json', orient='records', lines=True)

In [4]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'ImpoEner'
num_metric = 0
df_importaciones = extract_metric(date_dict,metric,num_metric)
df_importaciones

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,,2636.38000,2870.18000,2176.33000,1356.44000,1045.47000,1520.35000,405.18000,...,767.70000,786.68000,295.17000,,,1778.59000,2882.77000,698.75000,169.64000,2010-01-01
1,Sistema,Sistema,1905.68000,93.90000,1201.45000,135.00000,,,504.60000,1027.86000,...,,975.50000,1493.50000,,36.15000,1285.21000,1341.14000,2427.82000,2169.79000,2010-01-02
2,Sistema,Sistema,850.64000,63.40000,1223.48000,738.53000,293.86000,,,1614.73000,...,259.85000,285.20000,621.48000,,,2965.51000,1399.90000,1921.12000,1081.01000,2010-01-03
3,Sistema,Sistema,595.01000,609.62000,1162.19000,1073.98000,,809.62000,1260.99000,1663.19000,...,,,,,,,,,,2010-01-04
4,Sistema,Sistema,,,,,,,,,...,,,,,33.82000,2634.11000,,,,2010-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240,Sistema,Sistema,,,,,,,,,...,,,,161.06000,334.24000,313.85000,149.93000,,,2024-04-23
4241,Sistema,Sistema,,,,,,,,,...,,,,48.08000,80.00000,84.40000,76.57000,15.77000,,2024-04-24
4242,Sistema,Sistema,,,,,,,,,...,,,,,,1020.61000,2188.55000,1105.85000,,2024-04-28
4243,Sistema,Sistema,,,,,,,,,...,,1225.63000,,,598.55000,810.37000,1693.41000,3536.35000,,2024-04-29


In [5]:
df_importaciones.to_json('./data/importaciones.json', orient='records', lines=True)

## Variables de Precio

### Precio Bolsa Nacional

In [8]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'PrecBolsNaci'
num_metric = 0
df_precio_bolsa_nacional = extract_metric(date_dict,metric,num_metric)
df_precio_bolsa_nacional

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,124.68727,124.68727,124.28027,124.68727,124.68727,115.84427,114.68727,114.68727,...,115.84427,114.68727,124.68727,128.68727,129.68727,128.68727,124.68727,124.68727,124.68727,2010-01-01
1,Sistema,Sistema,114.50334,112.50334,112.50334,112.50334,112.50334,112.50334,112.50334,114.31134,...,115.50334,114.50334,114.50334,145.50334,150.50334,130.50334,130.50334,115.50334,115.50334,2010-01-02
2,Sistema,Sistema,112.79027,112.79027,112.79027,112.68727,112.68727,112.68727,97.68727,112.68727,...,113.68727,113.49527,113.68727,129.68727,199.68727,199.68727,119.68727,114.68727,114.68727,2010-01-03
3,Sistema,Sistema,112.58727,112.54127,112.54127,112.58727,112.58727,112.58727,112.58727,113.49527,...,115.68727,115.68727,113.68727,130.68727,305.68727,130.68727,129.68727,113.58727,112.58727,2010-01-04
4,Sistema,Sistema,113.58727,113.58727,113.58727,113.58727,113.58727,114.68727,114.68727,114.68727,...,117.68727,117.68727,117.68727,129.68727,209.68727,117.68727,117.68727,117.68727,114.68727,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,99.66307,101.66307,102.54807,102.54907,107.38807,174.38807,174.38807,199.45607,...,243.36807,254.36807,209.38807,513.38807,513.38807,364.38807,199.45607,199.45607,199.45607,2024-04-26
5230,Sistema,Sistema,101.53473,103.37573,106.37573,113.37573,113.37573,113.37573,113.37573,113.37573,...,225.68973,225.68973,225.68973,225.68973,225.68973,225.68973,113.37573,113.37573,113.37573,2024-04-27
5231,Sistema,Sistema,99.25669,102.14069,102.14169,102.14169,103.98169,103.98169,102.14169,102.14169,...,106.98169,109.98169,112.98169,113.98169,113.98169,113.98169,113.98169,113.98169,112.98169,2024-04-28
5232,Sistema,Sistema,99.14373,103.98373,105.98373,105.98373,105.98373,106.98373,109.98373,109.98373,...,191.70273,191.70273,191.70273,200.98373,230.98373,200.98373,191.70273,120.98373,110.98373,2024-04-29


In [9]:
df_precio_bolsa_nacional.to_json('./data/precio_bolsa_nacional.json', orient='records', lines=True)

### Precio Oferta Despacho

In [10]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'PrecOferDesp'
num_metric = 0
df_precio_oferta_despacho= extract_metric(date_dict,metric,num_metric)
df_precio_oferta_despacho

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,ALBG,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,...,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,2010-01-01
1,Recurso,CHBG,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,...,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,306.00000,2010-01-01
2,Recurso,CHVR,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,...,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,320.00000,2010-01-01
3,Recurso,CLMG,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,...,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2004.62100,2010-01-01
4,Recurso,CTG1,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,...,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,114.00000,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290667,Recurso,URA1,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,...,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,85.27600,2024-04-30
290668,Recurso,ZPA2,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,...,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,402.00000,2024-04-30
290669,Recurso,ZPA3,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,...,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,2024-04-30
290670,Recurso,ZPA4,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,...,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,385.00000,2024-04-30


In [11]:
df_precio_oferta_despacho.to_json('./data/precio_oferta_despacho.json', orient='records', lines=True)

### Maximo Precio Oferta Despacho

In [15]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'MaxPrecOferNal'
num_metric = 0
df_precio_oferta_despacho_maximo = extract_metric(date_dict,metric,num_metric)
df_precio_oferta_despacho_maximo

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,125.00000,125.00000,124.59300,125.00000,125.00000,116.15700,115.00000,115.00000,...,116.15700,115.00000,125.00000,129.00000,130.00000,129.00000,125.00000,125.00000,125.00000,2010-01-01
1,Sistema,Sistema,114.00000,112.00000,112.00000,112.00000,112.00000,112.00000,112.00000,113.80800,...,115.00000,114.00000,114.00000,145.00000,150.00000,130.00000,130.00000,115.00000,115.00000,2010-01-02
2,Sistema,Sistema,113.10300,113.10300,113.10300,113.00000,113.00000,113.00000,98.00000,113.00000,...,114.00000,113.80800,114.00000,130.00000,200.00000,200.00000,120.00000,115.00000,115.00000,2010-01-03
3,Sistema,Sistema,112.90000,112.85400,112.85400,112.90000,112.90000,112.90000,112.90000,113.80800,...,116.00000,116.00000,114.00000,131.00000,306.00000,131.00000,130.00000,113.90000,112.90000,2010-01-04
4,Sistema,Sistema,113.90000,113.90000,113.90000,113.90000,113.90000,115.00000,115.00000,115.00000,...,118.00000,118.00000,118.00000,130.00000,210.00000,118.00000,118.00000,118.00000,115.00000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,Sistema,Sistema,87.64810,89.64810,90.53310,90.53410,95.37310,162.37310,162.37310,187.44110,...,231.35310,242.35310,197.37310,501.37310,501.37310,352.37310,187.44110,187.44110,187.44110,2024-04-26
5206,Sistema,Sistema,90.53210,92.37310,95.37310,102.37310,102.37310,102.37310,102.37310,102.37310,...,214.68710,214.68710,214.68710,214.68710,214.68710,214.68710,102.37310,102.37310,102.37310,2024-04-27
5207,Sistema,Sistema,87.64810,90.53210,90.53310,90.53310,92.37310,92.37310,90.53310,90.53310,...,95.37310,98.37310,101.37310,102.37310,102.37310,102.37310,102.37310,102.37310,101.37310,2024-04-28
5208,Sistema,Sistema,90.53310,95.37310,97.37310,97.37310,97.37310,98.37310,101.37310,101.37310,...,183.09210,183.09210,183.09210,192.37310,222.37310,192.37310,183.09210,112.37310,102.37310,2024-04-29


In [16]:
df_precio_oferta_despacho_maximo.to_json('./data/precio_oferta_despacho_maximo.json', orient='records', lines=True)

### Precio de Escasez

In [4]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'PrecEsca'
num_metric = 0
df_precio_escasez= extract_metric(date_dict,metric,num_metric)
df_precio_escasez

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,356.70894,2010-01-01
1,Sistema,355.06286,2010-01-02
2,Sistema,355.07299,2010-01-03
3,Sistema,355.07299,2010-01-04
4,Sistema,354.78317,2010-01-05
...,...,...,...
5229,Sistema,742.65054,2024-04-26
5230,Sistema,742.65054,2024-04-27
5231,Sistema,742.65054,2024-04-28
5232,Sistema,742.65054,2024-04-29


In [5]:
df_precio_escasez.to_json('./data/precio_escasez.json', orient='records', lines=True)

## Variables de Consumo de Combustible

In [5]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'ConsCombustibleMBTU'
num_metric = 0
df_consumo_combustible_MBTU = extract_metric(date_dict,metric,num_metric)
df_consumo_combustible_MBTU

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_Name,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,ACPM,TVL1,284.62000,,,,,,,...,,20.43000,24.47000,468.46000,923.63000,855.99000,856.19000,855.61000,855.03000,2010-01-01
1,Recurso,CARBON,PPA1,419.36679,417.80781,423.22713,422.78171,422.78171,422.41052,422.26205,...,419.06984,421.89086,420.25764,421.81662,423.52408,421.37120,425.08307,427.97832,422.03934,2010-01-01
2,Recurso,CARBON,PPA2,872.69732,870.10314,875.57196,874.16969,873.95936,877.25467,872.76743,...,866.03658,866.24692,868.63076,874.87083,876.13286,874.59037,868.21008,870.80427,871.71574,2010-01-01
3,Recurso,CARBON,PPA3,856.27282,854.48562,862.18431,859.84721,861.70314,861.70314,860.12216,...,854.89805,855.58543,853.24833,861.29071,860.74081,858.47244,864.79636,865.20879,863.83403,2010-01-01
4,Recurso,CARBON,PPA4,1441.71161,1437.38985,1440.71428,1437.43734,1436.44001,1440.42933,1440.80926,...,1443.32633,1438.00724,1438.86209,1446.50829,1437.91226,1436.39252,1439.76444,1444.94106,1439.57447,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90487,Recurso,GAS,TYP4,454.17000,453.69000,454.12000,453.94000,454.10000,453.99000,453.07000,...,451.98000,452.15000,452.52000,454.44000,456.82000,457.33000,456.36000,455.93000,456.39000,2024-04-30
90488,Recurso,GAS NI,TBST,3738.59490,1009.08890,203.87090,,,,,...,3866.08460,3844.62610,4208.15870,5003.34050,5249.29460,5249.53430,5258.31950,5255.28710,5063.85960,2024-04-30
90489,Recurso,GAS NI,TCDT,4020.00000,4009.00000,4022.00000,4047.00000,4039.00000,4036.00000,4035.00000,...,4030.00000,3979.00000,4020.00000,4034.00000,3999.00000,4006.00000,3995.00000,4017.00000,3964.00000,2024-04-30
90490,Recurso,GAS NI,TFL1,1006.30952,1239.94064,1267.00676,1268.30318,1271.62525,1273.43856,1273.68144,...,1250.13061,1252.48414,1252.32183,1259.10335,1266.35855,1268.70831,1269.84267,1268.30318,1267.49291,2024-04-30


In [8]:
df_consumo_combustible_MBTU.to_json('./data/consumo_combustible_MBTU.json', orient='records', lines=True)

## Variables de Emisiones

In [4]:
date_dict = generate_date_dict(2017, 2024, 4)
metric = 'EmisionesCO2Eq'
num_metric = 0
df_emisiones_CO2eq = extract_metric(date_dict,metric,num_metric)
df_emisiones_CO2eq

Start for: (2017, 1): start_date = 2017-01-01, end_date = 2017-01-31
Done for: (2017, 1): start_date = 2017-01-01, end_date = 2017-01-31
Start for: (2017, 2): start_date = 2017-02-01, end_date = 2017-02-28
Done for: (2017, 2): start_date = 2017-02-01, end_date = 2017-02-28
Start for: (2017, 3): start_date = 2017-03-01, end_date = 2017-03-31
Done for: (2017, 3): start_date = 2017-03-01, end_date = 2017-03-31
Start for: (2017, 4): start_date = 2017-04-01, end_date = 2017-04-30
Done for: (2017, 4): start_date = 2017-04-01, end_date = 2017-04-30
Start for: (2017, 5): start_date = 2017-05-01, end_date = 2017-05-31
Done for: (2017, 5): start_date = 2017-05-01, end_date = 2017-05-31
Start for: (2017, 6): start_date = 2017-06-01, end_date = 2017-06-30
Done for: (2017, 6): start_date = 2017-06-01, end_date = 2017-06-30
Start for: (2017, 7): start_date = 2017-07-01, end_date = 2017-07-31
Done for: (2017, 7): start_date = 2017-07-01, end_date = 2017-07-31
Start for: (2017, 8): start_date = 2017-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,2S6S,,,,,,,,,...,,,,,,,,,0.04264,2018-01-01
1,Recurso,2S8G,,,,,,,,0.00581,...,1.28771,1.30005,1.32917,1.33504,1.31883,1.26893,1.28473,1.34893,1.42599,2018-01-01
2,Recurso,2S8I,3.48035,3.48035,3.27153,3.20192,3.09751,3.13232,3.41074,3.48035,...,2.67987,2.92349,2.67987,2.85389,2.88869,2.71467,2.64507,2.33183,2.36664,2018-01-01
3,Recurso,2S8S,0.00051,0.00051,0.00051,0.00051,0.00051,0.00042,0.00035,0.00052,...,0.00052,0.00052,0.00052,0.00052,0.00052,0.00052,0.00052,0.00052,0.00052,2018-01-01
4,Recurso,2S9Q,0.00001,,,,,,,,...,,,,,,,,,,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107003,Recurso,TYP4,26.63901,26.61086,26.63608,26.62552,26.63490,26.62845,26.57449,26.50410,...,26.51056,26.52053,26.54223,26.65485,26.79444,26.82436,26.76746,26.74224,26.76922,2024-04-30
107004,Recurso,ZPA2,40.88586,40.93101,40.87369,40.91088,40.83296,40.91561,40.83532,40.93053,...,40.39339,40.38576,40.42025,40.41129,40.26377,39.35579,40.04325,40.48024,40.50421,2024-04-30
107005,Recurso,ZPA3,,,,,,,,,...,,,,,,3.39046,21.18428,30.07950,37.04601,2024-04-30
107006,Recurso,ZPA4,77.37187,77.45489,77.53907,77.43788,77.85533,77.85208,77.94178,77.99733,...,78.57200,78.27978,78.42909,78.14796,78.55359,78.42850,77.84566,77.87681,78.58427,2024-04-30


In [6]:
df_emisiones_CO2eq.to_json('./data/emisiones_CO2eq.json', orient='records', lines=True)